In [1]:
'''
Calculate significantly enriched pathways of selected 91 CLUE compounds
'''
import pandas as pd
import os
from Enrichment import *
from collections import Counter
import numpy as np
import scipy.cluster.hierarchy as sch
from scipy.stats import pearsonr,spearmanr
from scipy.spatial.distance import cosine


In [2]:
# set the correct working directory
if os.path.isdir('M:/'):
    os.chdir('M:\Box\Jake-Jegga\IPF Drug Discovery\Results')
elif os.path.isdir('E:/'):
    os.chdir('E:/Box Sync/Jake-Jegga/IPF Drug Discovery/Results')
else:
    os.chdir('/Users/wano3m/Box Sync/Jake-Jegga/IPF Drug Discovery/Results')
enrich = SEA('../../../Ontology_Info/')
termids = pd.read_table('../../../Lincs_data/TermID2TermName.txt',index_col = 0)
master_gene_table = pd.read_csv('../../../Ontology_Info/Master Gene Conversion Table.csv',dtype = str)
master_gene_table = master_gene_table.dropna(subset=['h_entrez_id']).set_index('h_entrez_id')

In [ ]:
# enrichment of 6 ipf DEG datasets
ipf_deg = pd.read_excel('../Intermediate results/Deg lists included in paper/IPF DEG FC06 pval005 protein coding DEG.xlsx',index_col = 0)
calculating enrichment score
refs = ['BP','Pathway','MP']
enrichment_report = pd.DataFrame()
for lib in refs:
    lib_enrichment_results = pd.DataFrame()
    ref = enrich.dict_g2t[lib]
    M = 19055
    for dataset in ipf_deg.columns[:-2]:
        gl = ipf_deg.ix[:,dataset].dropna().values
        gl = [[x] if ' /// ' not in x else x.split(' /// ') for x in gl]
        gl = list(set([item for sublist in gl for item in sublist]))[:500]
        enrichment_result = enrich.get_pvalues(gl,ref,M,gene_type='H_symbol')[2].Raw_p_value
        enrichment_result.name = dataset
        lib_enrichment_results = pd.concat([lib_enrichment_results,enrichment_result],axis = 1)
    enrichment_report = enrichment_report.append(lib_enrichment_results)
simple_ER = abs(np.log10(enrichment_report.copy().fillna(1)))
simple_ER.insert(0,'Term_name',termids.ix[simple_ER.index,0])
simple_ER.to_excel('./IPF datasets enrichment.xlsx')

In [ ]:
# make summary logfc heatmap of 6 ipf datasets
all_deg = []
for dataset in ipf_deg.columns[:-2]:
    gl = ipf_deg.ix[:,dataset].dropna().values.tolist()
    all_deg += gl
all_deg = list(set(all_deg))
full_deg_path = '../Intermediate results/Deg lists included in paper'
full_deg_fns = [x for x in os.listdir(full_deg_path) if 'full' in x ]
df_logfc = pd.DataFrame()
for fn in full_deg_fns:
    dataset = fn.replace(' DEG full report.txt','')
    fn = os.path.join(full_deg_path,fn)
    tmp = pd.read_table(fn,index_col=0).loc[all_deg,'logFC'].dropna()
    tmp = tmp.groupby(tmp.index,sort=False).median()
    tmp.name = dataset
    df_logfc = pd.concat((df_logfc,tmp),axis = 1)
df_logfc.fillna(0,inplace=True)
df_logfc.to_excel('Gene logFC in 6 datasets.xlsx')

In [ ]:
if 'Enrichment analysis' not in os.getcwd():
    os.chdir('../Intermediate results/Enrichment analysis')
mea_up = pd.DataFrame()
mea_dn = pd.DataFrame()
for i in range(2):
    if i == 0:
        gbyt = 'IPF enrichment up.gct'
        pval = 'IPF metaanalysis median FC 06 enrichment.csv'
        out_put = mea_up
    else:
        gbyt = 'IPF enrichment dn.gct'
        pval = 'IPF metaanalysis median FC -06 enrichment.csv'
        out_put = mea_dn
    gbyt = pd.read_table(gbyt,index_col=0,skiprows=2)
    termbycluster = gbyt.ix[:,0].copy()
    km_groups = gbyt.groupby('k_means_10').mean()
    for ind in km_groups.index:
        gene_vector = km_groups.loc[ind]
        gene_vector = gene_vector[gene_vector>0]
        genes = gene_vector.sort_values(ascending=False).index[:int(0.2*len(gene_vector))].tolist()
        terms = termbycluster[termbycluster == ind].index
        best_terms = gbyt.ix[terms,1:].transpose().corrwith(gene_vector).sort_values(ascending = False).index.tolist()
        best_disease = [x for x in best_terms if 'Disease_' in x][0]
        best_terms = [x for x in best_terms if ('Disease_' not in x) & ('HP_' not in x)][:2] + [best_disease]
        out_put.ix['KM_'+str(ind),'Genes'] = ', '.join(genes)
        out_put.ix['KM_'+str(ind),'Terms'] = ', '.join(best_terms)
    out_put.to_csv('IPF_MEA.csv',mode = 'a')

In [41]:
# Improved version using grouping
# load Lincs data

# lincs_up_gl = pd.read_table('../../../Lincs_data/Lincs_Cmpd_Up_500_Genes.txt',index_col = 0)
# lincs_dn_gl = pd.read_table('../../../Lincs_data/Lincs_Cmpd_Dn_500_Genes.txt',index_col = 0)
# lincs_up_gl['reg'] = 'up'
# lincs_dn_gl['reg'] = 'dn'
# lincs_gl = lincs_up_gl.append(lincs_dn_gl)
# del lincs_up_gl
# del lincs_dn_gl
# lincs_meta_data = pd.read_table('../../../Lincs_data/Lincs_P1+P2_sig_info.txt',index_col = 0)
# all_lincs_genes = pd.read_table('../../../Lincs_data/GSE92742_Broad_LINCS_gene_info.txt').ix[:,0].tolist()

'''
Find conserverd, uniformly regulated genes in the cellline that showed significant connectivity. 
Then calculated enrichment in the biological process, pathway and mouse phenotype categories.
'''

def _get_conserved_genes(df,t):
    global dose_sep
    allgenes = list(set(','.join(df).split(',')))
    gene_counts = pd.DataFrame(0,index = allgenes,columns = ['dn','up'])
    for i in range(2):
        genes = df[i].split(',')
        if dose_sep:
            reg = df.index[i][2]
        else:
            reg = df.index[i][1]
        gene_count_tmp = Counter(genes)
        gene_count_tmp = pd.Series(gene_count_tmp)
        gene_counts.ix[gene_count_tmp.index,reg] = gene_count_tmp.values
    gene_counts['score'] = gene_counts.up-gene_counts.dn
    common_up_genes = gene_counts.index[gene_counts.score >= threshold].tolist()
    common_dn_genes = gene_counts.index[gene_counts.score <= -threshold].tolist()
    return gene_counts, common_up_genes, common_dn_genes    

refs = ['BP','Pathway','MP']
enrichment_report = pd.DataFrame()
for cmpd in cmpds:
#     cmpd_lincs_gl = lincs_gl[(lincs_gl.pert_iname==cmpd) & (lincs_gl.pert_itime == '24 h')]
    cmpd_lincs_gl = lincs_gl[lincs_gl.pert_iname==cmpd]
    if cmpd_lincs_gl.shape[0] == 0:
        continue
    print('Processing compound {}'.format(cmpd))
    grouping_vector = ['cell_id','pert_idose']+ ['reg']
    lincs_gl_grouped = cmpd_lincs_gl.groupby(grouping_vector).Top_genes.agg(lambda x: ','.join(x))
    counts = cmpd_lincs_gl.groupby(grouping_vector).reg.count().values
    counts = [x for i,x in enumerate(counts) if i%2==0]

    for lib in refs:
        print('Calculating enrichment for {}'.format(lib))
        lib_enrichment_results = pd.DataFrame()
        ref = enrich.dict_g2t[lib]
        M = len(set(ref.index.astype(int).tolist()+all_lincs_genes))
        if 'pert_idose' in grouping_vector:
            dose_sep = True
            iterator = lincs_gl_grouped.groupby(level=[0,1])
        else:
            dose_sep = False
            iterator = lincs_gl_grouped.groupby(level=0)
        for df,profile_counts in zip(iterator,counts):
            metadata = df[0]
            if isinstance(metadata,str):
                metadata = [metadata]
            df = df[1]
            threshold = 2/3*profile_counts
            cm_up,cm_dn =_get_conserved_genes(df,threshold)[1:]
            print('Calculating Enrichment for {} from {} profiles'.format(metadata,profile_counts))
            for query, reg in zip([cm_up,cm_dn],['up','dn']):
                query = [x for x in query if x!= '']
                if len(query) <=50:
                    print('\t {} regulated Query has less than 50 genes, skipped'.format(reg))
                    continue             
                print('\tCalculating enrichment from {} genes'.format(str(len(query))))
                tmp_enrichment_result = enrich.get_pvalues(query,ref,M,return_gene_info=True)[2].ix[:,1:]
                if tmp_enrichment_result.shape[0] == 0:
                    continue
                tmp_enrichment_result['cmpd'] = cmpd
                if len(metadata[0]) ==1:
                    raise Exception
                tmp_enrichment_result['cellline'] = metadata[0]
                if dose_sep:
                    tmp_enrichment_result['dose'] = metadata[1]
                tmp_enrichment_result['reg'] = reg
                tmp_enrichment_result.set_index('cmpd',inplace = True)
                query_gene_header = pd.DataFrame(tmp_enrichment_result.ix[0,[2,3,5,6]]).transpose()
                query_gene_header.genelist = ', '.join(sorted(master_gene_table.loc[query,'h_symbol'].dropna().unique()))
                query_gene_header.num_query = len(query)
                tmp_enrichment_result = query_gene_header.append(tmp_enrichment_result)
                # combine enrichment results from BP, MP and pathway            
                enrichment_report = pd.concat([enrichment_report,tmp_enrichment_result])
enrichment_report.cellline = enrichment_report.cellline + "_" + enrichment_report.dose
enrichment_report.drop('dose',axis = 1).to_csv('91+nintedanib enrichment by dose by cell type report.txt',sep = '\t')

Processing compound alclometasone
Calculating enrichment for BP
Calculating Enrichment for ('A375', '10 µM') from 2 profiles
	 up regulated Query has less than 50 genes, skipped
	 dn regulated Query has less than 50 genes, skipped
Calculating Enrichment for ('A549', '10 µM') from 2 profiles
	Calculating enrichment from 57 genes
	Calculating enrichment from 70 genes
Calculating Enrichment for ('ASC', '10 µM') from 1 profiles
	Calculating enrichment from 500 genes
	Calculating enrichment from 500 genes
Calculating Enrichment for ('HA1E', '10 µM') from 2 profiles
	Calculating enrichment from 54 genes
	 dn regulated Query has less than 50 genes, skipped
Calculating Enrichment for ('HCC515', '10 µM') from 2 profiles
	Calculating enrichment from 73 genes
	Calculating enrichment from 75 genes
Calculating Enrichment for ('HEPG2', '10 µM') from 2 profiles
	Calculating enrichment from 59 genes
	 dn regulated Query has less than 50 genes, skipped
Calculating Enrichment for ('HT29', '10 µM') from 